In [410]:
# start_date = '2025-11-01'
# end_date   = '2025-11-23'

In [411]:
start_date = '2026-01-01'
end_date   = '2026-01-12'

In [412]:
import warnings
warnings.filterwarnings("ignore", message=".*SQLAlchemy connectable.*")

In [413]:
import pandas as pd
import numpy as np
import re

#### Server Connection

In [414]:
import json, os
import pyodbc
with open(os.path.join(os.path.expanduser("~"), ".my_creds.json")) as f:
    creds = json.load(f)

db_user = creds["db_user"]
db_pass = creds["db_pass"]
server = "157.119.230.120,4071"
database = "PowerBI_LISV2"
#connection string
conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={db_user};
PWD={db_pass};
TrustServerCertificate=yes;
"""

conn = pyodbc.connect(conn_str)
print("Server Connected")

Server Connected


### StoreAttendance

In [415]:
StoreAttendanceQuery=f"""
SELECT K.UserName,K.FullName,K.LISStoreCode,K.StoreName,K.VisitDate,K.PJPStatus,K.[CheckIn-Time], K.[CheckOut-Time],
	K.TimeDuration,
    K.AutoCheckOutStatus,K.NoPermissionReason,K.[StoreCheck-InImageLink],K.[StoreCheck-OutImageLink],K.[FR Image],
    K.NoPermissionImageLink,K.AttendanceType 
FROM(
SELECT UserName,FullName,LISStoreCode,StoreName,VisitDate,PJPStatus,[CheckIn-Time], [CheckOut-Time],TimeDuration,
    AutoCheckOutStatus,NoPermissionReason,[StoreCheck-InImageLink],[StoreCheck-OutImageLink],Null as 'FR Image',
    NoPermissionImageLink,AttendanceType,
	RANK() OVER (PARTITION BY TRY_CONVERT(date, [VisitDate], 105),LISStoreCode ORDER BY AttendanceType ASC) AS RK
FROM PowerBI_LISV2.DaburPharmaManagement.ISPStoreAttendance_NormalExport
WHERE 
    UserName NOT LIKE '%SUP%' 
	-- AND LISStoreCode = 'DBRPH-PNV0MH1014135'
    AND LOWER(UserName) NOT LIKE '%test%'
    AND LOWER(UserName) NOT LIKE '%star%'
    AND [Month] = 11
    AND [Year] = 2025
    AND TRY_CONVERT(date, [VisitDate], 105) >= '{start_date}'
    AND TRY_CONVERT(date, [VisitDate], 105) <= '{end_date}'
	) AS K
	WHERE K.RK =1;

"""

In [416]:
StoreAttendance= pd.read_sql(StoreAttendanceQuery,conn)

In [417]:
StoreAttendance.columns

Index(['UserName', 'FullName', 'LISStoreCode', 'StoreName', 'VisitDate',
       'PJPStatus', 'CheckIn-Time', 'CheckOut-Time', 'TimeDuration',
       'AutoCheckOutStatus', 'NoPermissionReason', 'StoreCheck-InImageLink',
       'StoreCheck-OutImageLink', 'FR Image', 'NoPermissionImageLink',
       'AttendanceType'],
      dtype='object')

In [418]:
# Convert to datetime.time format and back to string in HH:MM:SS
StoreAttendance["CheckIn-Time"] = pd.to_datetime(StoreAttendance["CheckIn-Time"], format="%H:%M:%S").dt.strftime("%H:%M:%S")
StoreAttendance["CheckOut-Time"] = pd.to_datetime(StoreAttendance["CheckOut-Time"], format="%H:%M:%S").dt.strftime("%H:%M:%S")

In [419]:
StoreAttendance["Covered"] = "Y"
# StoreAttendance["Merchandised"] = np.where(
#     StoreAttendance["NoPermissionReason"].isna() | (StoreAttendance["NoPermissionReason"] == ""),
#     "Y",
#     "N"
# )

StoreAttendance["Merchandised"] = np.where(
    StoreAttendance["AttendanceType"] == "NoPermission",
    "N",
    "Y"
)

In [420]:
# Convert to datetime (if not already)
# StoreAttendance["VisitDate"] = pd.to_datetime(StoreAttendance["VisitDate"], errors="coerce")

# # Extract only the date part (keeps as datetime.date)
# StoreAttendance["VisitDate"] = StoreAttendance["VisitDate"].dt.date
# StoreAttendance["VisitDate"] = StoreAttendance["VisitDate"].apply(lambda x: x.strftime("%d-%m-%Y"))

In [421]:
StoreAttendance.to_csv('StoreAttendance.csv')

In [422]:
StoreAttendance.head()

Empty DataFrame
Columns: [UserName, FullName, LISStoreCode, StoreName, VisitDate, PJPStatus, CheckIn-Time, CheckOut-Time, TimeDuration, AutoCheckOutStatus, NoPermissionReason, StoreCheck-InImageLink, StoreCheck-OutImageLink, FR Image, NoPermissionImageLink, AttendanceType, Covered, Merchandised]
Index: []

In [423]:
# start_date = '2025-11-01'
# end_date   = '2025-11-09'

In [424]:
PrimaryShelfQuery = f"""
SELECT DISTINCT
    VisitID,
    TRY_CONVERT(date, [SubmittedDate&Time], 105) AS SubmittedDate,
    Username,FullName,Designation,LISStoreCode,ProductCode,ProductName,
    Quantity,Amount,
    (CASE WHEN Quantity >=3  THEN 1 ELSE 0 END) AS OSA
FROM PowerBI_LISV2.DaburPharmaManagement.PrimaryShelfStockReportExport
WHERE 
    UserName NOT LIKE '%SUP%'
    AND LOWER(UserName) NOT LIKE '%test%'
    AND LOWER(UserName) NOT LIKE '%star%'
    AND [Month] = 11
    AND [Year] = 2025
    AND TRY_CONVERT(date, [SubmittedDate&Time], 105) >= '{start_date}'
    AND TRY_CONVERT(date, [SubmittedDate&Time], 105) <= '{end_date}'
ORDER BY 
    TRY_CONVERT(date, [SubmittedDate&Time], 105);
"""


In [425]:
PrimaryShelf = pd.read_sql(PrimaryShelfQuery,conn)

In [426]:
PrimaryShelf = PrimaryShelf.drop_duplicates()

In [427]:
PrimaryShelfOSA = (
    PrimaryShelf.groupby(["VisitID", "SubmittedDate", "Username", "LISStoreCode"], dropna=False)
      .agg(**{
          "OSA Target": ("OSA", "size"),
          "OSA Avail": ("OSA", "sum"),
      })
      .reset_index()
)

In [428]:
# Convert to datetime (auto-detects multiple formats)
PrimaryShelfOSA["SubmittedDate"] = pd.to_datetime(PrimaryShelfOSA["SubmittedDate"], errors="coerce")

# Format into dd-mm-yyyy string
PrimaryShelfOSA["SubmittedDate"] = PrimaryShelfOSA["SubmittedDate"].dt.strftime("%d-%m-%Y")

In [429]:
PrimaryShelfOSA.head()

Empty DataFrame
Columns: [VisitID, SubmittedDate, Username, LISStoreCode, OSA Target, OSA Avail]
Index: []

In [430]:
PrimaryShelfOSA.to_csv('PrimaryShelfOSA.csv')

In [431]:
StoreAttendance['VisitDate'] = pd.to_datetime(StoreAttendance['VisitDate'], errors='coerce', dayfirst=True).dt.normalize()
PrimaryShelfOSA['SubmittedDate'] = pd.to_datetime(PrimaryShelfOSA['SubmittedDate'], errors='coerce', dayfirst=True).dt.normalize()

In [432]:
# StoreAttendance.head()

In [433]:
PrimaryShelfOSA = PrimaryShelfOSA.add_prefix("PS_") # renaming column to add prefix

In [434]:
PrimaryShelfOSA.columns

Index(['PS_VisitID', 'PS_SubmittedDate', 'PS_Username', 'PS_LISStoreCode',
       'PS_OSA Target', 'PS_OSA Avail'],
      dtype='object')

In [435]:
pss1 = pd.merge(StoreAttendance,PrimaryShelfOSA[['PS_SubmittedDate', 'PS_Username', 'PS_LISStoreCode','PS_OSA Target','PS_OSA Avail']], left_on =['VisitDate','UserName','LISStoreCode'], 
                right_on=['PS_SubmittedDate', 'PS_Username', 'PS_LISStoreCode'], how = 'left')

In [436]:
pss1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   UserName                 0 non-null      object        
 1   FullName                 0 non-null      object        
 2   LISStoreCode             0 non-null      object        
 3   StoreName                0 non-null      object        
 4   VisitDate                0 non-null      datetime64[ns]
 5   PJPStatus                0 non-null      object        
 6   CheckIn-Time             0 non-null      object        
 7   CheckOut-Time            0 non-null      object        
 8   TimeDuration             0 non-null      object        
 9   AutoCheckOutStatus       0 non-null      object        
 10  NoPermissionReason       0 non-null      object        
 11  StoreCheck-InImageLink   0 non-null      object        
 12  StoreCheck-OutImageLink  0 non-null      object 

In [437]:
# pss1.to_csv('pss1.csv')

#### SOS ( Data Preparation , Unpivot, Pivot & ColumnReordering

In [438]:
SOSQuery =f"""
SELECT 
    VisitId,
    TRY_CONVERT(date, [SubmittedDate&Time], 105) AS SubmittedDate,
    UserName,FullName,LISStoreCode,IscategoryAvailable,ReasonforNonavailability,
    CategoryTrackingName,ISPCategoryTrackingProductCode,CategoryProductName,
    Facings,TotalFacings,FacingContribution,TgtCatSOS,
    ImageLink as 'URL'
FROM PowerBI_LISV2.DaburPharmaManagement.CategoryTrackingVisitExport
WHERE 
    UserName NOT LIKE '%SUP%'
    AND LOWER(UserName) NOT LIKE '%test%'
    AND LOWER(UserName) NOT LIKE '%star%'
    AND [Month] = 11
    AND [Year] = 2025
    AND TRY_CONVERT(date, [SubmittedDate&Time], 105) >= '{start_date}'
    AND TRY_CONVERT(date, [SubmittedDate&Time], 105) <= '{end_date}'
ORDER BY 
    TRY_CONVERT(date, [SubmittedDate&Time], 105);

"""

In [439]:
SOSData = pd.read_sql(SOSQuery,conn)

In [440]:
SOSData= SOSData.rename(columns = {'Facings':'DaburFacings'})

In [441]:
SOSData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   VisitId                         0 non-null      object
 1   SubmittedDate                   0 non-null      object
 2   UserName                        0 non-null      object
 3   FullName                        0 non-null      object
 4   LISStoreCode                    0 non-null      object
 5   IscategoryAvailable             0 non-null      object
 6   ReasonforNonavailability        0 non-null      object
 7   CategoryTrackingName            0 non-null      object
 8   ISPCategoryTrackingProductCode  0 non-null      object
 9   CategoryProductName             0 non-null      object
 10  DaburFacings                    0 non-null      object
 11  TotalFacings                    0 non-null      object
 12  FacingContribution              0 non-null      object
 13  Tg

In [442]:
# Correct filtering: one boolean condition inside one pair of brackets
mask = SOSData["TgtCatSOS"].notna() & (SOSData["TgtCatSOS"].astype(str).str.strip() != "")
SOSData = SOSData[mask]

In [443]:
cols_to_replace_null = ["TotalFacings", "DaburFacings", "FacingContribution"]
SOSData[cols_to_replace_null] = SOSData[cols_to_replace_null].fillna(0)

In [444]:
SOSData["FacingContribution"] = pd.to_numeric(SOSData["FacingContribution"], errors="coerce")
SOSData["TgtCatSOS"] = pd.to_numeric(SOSData["TgtCatSOS"], errors="coerce")

SOSData["IsSOSAchieved"] = np.where(
    SOSData["FacingContribution"] >= SOSData["TgtCatSOS"],
    1, 
    0
).astype("int64")

In [445]:
# SOSData["IscategoryAvailable"] = np.where(
#     SOSData["IscategoryAvailable"] == 1,
#     'YES', 
#     'NO'
# ).astype(str)

In [446]:
SOSData.columns

Index(['VisitId', 'SubmittedDate', 'UserName', 'FullName', 'LISStoreCode',
       'IscategoryAvailable', 'ReasonforNonavailability',
       'CategoryTrackingName', 'ISPCategoryTrackingProductCode',
       'CategoryProductName', 'DaburFacings', 'TotalFacings',
       'FacingContribution', 'TgtCatSOS', 'URL', 'IsSOSAchieved'],
      dtype='object')

In [447]:
SOSData.to_csv('SOSData.csv') # chek here for duplication at date x user x store x catetracking name - can user re submit the facing counts 

In [448]:
# Select only specific columns from your DataFrame
ColumnsForPivot = SOSData[
    [
        "SubmittedDate",
        "UserName",
        "LISStoreCode",
        "IscategoryAvailable",
        "CategoryTrackingName",
        "DaburFacings",
        "TotalFacings",
        "FacingContribution",
        "TgtCatSOS",
        "URL",
        "IsSOSAchieved"
    ]
]

Unpivoted_SOSData = pd.melt(
    ColumnsForPivot,
    id_vars=[
        "SubmittedDate",
        "UserName",
        "LISStoreCode",
        "CategoryTrackingName"
    ],
    var_name="Attribute",
    value_name="Value"
)



In [449]:
Unpivoted_SOSData = Unpivoted_SOSData.sort_values(
    by=["SubmittedDate", "UserName", "LISStoreCode", "CategoryTrackingName"]
).reset_index(drop=True)

In [450]:
# Create new column by combining CategoryTrackingName and Attribute with an underscore
Unpivoted_SOSData["Category_Attribute"] = (
    Unpivoted_SOSData["CategoryTrackingName"].astype(str).str.strip() + "_" +
    Unpivoted_SOSData["Attribute"].astype(str).str.strip()
)

In [451]:
Unpivoted_SOSData.head()

Empty DataFrame
Columns: [SubmittedDate, UserName, LISStoreCode, CategoryTrackingName, Attribute, Value, Category_Attribute]
Index: []

In [452]:
Unpivoted_SOSData= Unpivoted_SOSData.drop(columns = ['CategoryTrackingName','Attribute'])

In [453]:
SOSDatadedup = Unpivoted_SOSData.drop_duplicates(
    subset=["SubmittedDate", "UserName", "LISStoreCode", "Category_Attribute"],
    keep="first"
)

In [454]:
Pivoted_SOS_Data = (
    SOSDatadedup.pivot_table(
        index=["SubmittedDate", "UserName", "LISStoreCode"],
        columns="Category_Attribute",
        values="Value",
        aggfunc="first"          
    )
    .reset_index()
)

In [455]:
Pivoted_SOS_Data.head()

Empty DataFrame
Columns: [SubmittedDate, UserName, LISStoreCode]
Index: []

In [456]:
# list(Pivoted_SOS_Data.columns)

In [457]:
# Pivoted_SOS_Data = Pivoted_SOS_Data[ordered_cols]

In [458]:
Pivoted_SOS_Data.head()

Empty DataFrame
Columns: [SubmittedDate, UserName, LISStoreCode]
Index: []

In [459]:
Pivoted_SOS_Data.to_csv("Pivoted_SOS_Data.csv")

In [460]:
sos_cols = [col for col in Pivoted_SOS_Data.columns if "IsSOSAchieved" in col]
Pivoted_SOS_Data["SumSOSAchieved"] = Pivoted_SOS_Data[sos_cols].sum(axis=1, skipna=True)

In [461]:
sos_cols = [col for col in Pivoted_SOS_Data.columns if "Iscategoryavailable" in col]
Pivoted_SOS_Data["SumIsCategoryAvailable"] = Pivoted_SOS_Data[sos_cols].sum(axis=1, skipna=True)

In [462]:
cols = list(Pivoted_SOS_Data.columns)
order_pattern = [
    'IscategoryAvailable',
    'DaburFacings',
    'TotalFacings',
    'FacingContribution',
    'TgtCatSOS',
    'IsSOSAchieved',
    'URL'
]
# Extract unique categories
categories = sorted({re.split(r'_', c, 1)[0] for c in cols if '_' in c})

# Build ordered list
ordered_cols = ['SubmittedDate', 'UserName', 'LISStoreCode','SumSOSAchieved','SumIsCategoryAvailable']
for cat in categories:
    for suffix in order_pattern:
        match = f"{cat}_{suffix}"
        if match in cols:
            ordered_cols.append(match)

In [463]:
Pivoted_SOS_Data = Pivoted_SOS_Data[ordered_cols]

In [464]:
Pivoted_SOS_Data.info() # SOS Data Cleaned and Reordered

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SubmittedDate           0 non-null      object 
 1   UserName                0 non-null      object 
 2   LISStoreCode            0 non-null      object 
 3   SumSOSAchieved          0 non-null      float64
 4   SumIsCategoryAvailable  0 non-null      float64
dtypes: float64(2), object(3)
memory usage: 132.0+ bytes


#### Combining pss1 with SOS Cleaned & Pivoted Dataset

In [465]:
Pivoted_SOS_Data['SubmittedDate'] = pd.to_datetime(Pivoted_SOS_Data['SubmittedDate'], errors='coerce', dayfirst=True).dt.normalize()

In [466]:
Pivoted_SOS_Data = Pivoted_SOS_Data.add_prefix("SOS_") # renaming column to add prefix

In [467]:
pss2 = pd.merge(pss1,Pivoted_SOS_Data,left_on = ['LISStoreCode', 'UserName', 'VisitDate'], 
                right_on=['SOS_LISStoreCode', 'SOS_UserName', 'SOS_SubmittedDate'],how='left')

In [468]:
pss2.head(2)

Empty DataFrame
Columns: [UserName, FullName, LISStoreCode, StoreName, VisitDate, PJPStatus, CheckIn-Time, CheckOut-Time, TimeDuration, AutoCheckOutStatus, NoPermissionReason, StoreCheck-InImageLink, StoreCheck-OutImageLink, FR Image, NoPermissionImageLink, AttendanceType, Covered, Merchandised, PS_SubmittedDate, PS_Username, PS_LISStoreCode, PS_OSA Target, PS_OSA Avail, SOS_SubmittedDate, SOS_UserName, SOS_LISStoreCode, SOS_SumSOSAchieved, SOS_SumIsCategoryAvailable]
Index: []

[0 rows x 28 columns]

In [469]:
# pss2.to_csv('pss2.csv')

Now onwards BayBranding & Paid Visibility , Wvisibility will be merged with PSS2 once cleaning done

#### Paid Visibility ( Bay Branding & Window Visibility ) 

#### A. Paid Visibility ( Window)

In [470]:
PaidVisibilityQuery = f"""
SELECT DISTINCT
    TRY_CONVERT(date, [SubmittedDateandTime], 105) AS SubmittedDate,
    UserName,LISStoreCode,TRIM(POSMBrand) AS POSMBrand,POSMName,VisibilityAvailable,CloseUpPhotoLink AS URL
    
FROM 
    PowerBI_LISV2.DaburPharmaManagement.PaidVisibilityReportExport
WHERE 
    MONTH = 11 
    AND YEAR = 2025  
    -- AND AppQuestions IS NOT NULL 
    -- AND AppQuestions != ''
    AND UserName NOT LIKE '%SUP%'
    AND LOWER(UserName) NOT LIKE '%test%'
    AND LOWER(UserName) NOT LIKE '%star%'
    -- AND POSMName = 'Window'
    AND TRY_CONVERT(date, [SubmittedDateandTime], 105) >= '{start_date}'
    AND TRY_CONVERT(date, [SubmittedDateandTime], 105) <= '{end_date}'
ORDER BY 
    SubmittedDate;

  ;
"""

In [471]:
PaidVisibilityQuery = pd.read_sql(PaidVisibilityQuery,conn)

In [472]:
# WindowVisibilityData.to_csv('PaidVisibilityAuditInputReport.csv')

In [473]:
PaidVisibilityQuery.to_csv(
    'PaidVisibilityAuditInputReport.csv',
    index=False,
    encoding='utf-8'
)


In [474]:
PaidVisibilityQuery.head(2)

Empty DataFrame
Columns: [SubmittedDate, UserName, LISStoreCode, POSMBrand, POSMName, VisibilityAvailable, URL]
Index: []

In [475]:
WindowVisibilityData = PaidVisibilityQuery[PaidVisibilityQuery['POSMName']=='Window']

In [476]:
selected_columns = [
    "SubmittedDate",
    "UserName",
    "LISStoreCode",
    "POSMBrand",
    "VisibilityAvailable",
    "URL"
]

In [477]:
def select_columns(df, cols):
    # keep only columns that actually exist (avoids KeyError if column missing)
    cols_present = [c for c in cols if c in df.columns]
    return df.loc[:, cols_present].copy()

In [478]:
WindowVisibilityData = WindowVisibilityData.drop(columns=['POSMName'])
SelectedDf = select_columns(WindowVisibilityData, selected_columns)

In [479]:
ColumnsForPivot = WindowVisibilityData.copy()

In [480]:
# The id_vars (columns to keep) are:
id_vars = ["SubmittedDate", "UserName", "LISStoreCode", "POSMBrand"]

In [481]:
def unpivot_other_columns(df, id_vars, attr_name="Attribute", val_name="Value"):
    # keep only id_vars that exist
    
    id_vars_present = [c for c in id_vars if c in df.columns]
    
    value_vars = [c for c in df.columns if c not in id_vars_present] # value_vars are all columns except the id_vars_present
    # pandas.melt performs the 'unpivot'
    melted = df.melt(id_vars=id_vars_present, value_vars=value_vars,
                     var_name=attr_name, value_name=val_name)
    return melted

In [482]:
WindowVisibilityDataUnpivotedDf = unpivot_other_columns(ColumnsForPivot, id_vars, attr_name="Attribute", val_name="Value")

In [483]:
WindowVisibilityDataUnpivotedDf.head()

Empty DataFrame
Columns: [SubmittedDate, UserName, LISStoreCode, POSMBrand, Attribute, Value]
Index: []

In [484]:
WindowVisibilityDataUnpivotedDf["Brand_Attribute"] = (
    WindowVisibilityDataUnpivotedDf["POSMBrand"].astype(str) + "_" + WindowVisibilityDataUnpivotedDf["Attribute"].astype(str)
)


In [485]:
WindowVisibilityDataUnpivotedDf = WindowVisibilityDataUnpivotedDf.drop(columns = ['Attribute','POSMBrand']) 

In [486]:
group_cols = ["SubmittedDate","UserName","LISStoreCode","Brand_Attribute"]

WindowVisibilityDataUnpivotedDf_grouped = (
    WindowVisibilityDataUnpivotedDf
        .groupby(group_cols, as_index=False)["Value"]
        .first()
)

In [487]:
WindowVisibilityDataUnpivotedDf_grouped.head()

Empty DataFrame
Columns: [SubmittedDate, UserName, LISStoreCode, Brand_Attribute, Value]
Index: []

In [488]:
pivot_index = ["SubmittedDate", "UserName", "LISStoreCode"]

WindowVisibilityDataPivoted = (
    WindowVisibilityDataUnpivotedDf_grouped
        .pivot_table(
            index=pivot_index,
            columns="Brand_Attribute",
            values="Value",
            aggfunc="first"
        )
        .reset_index()
)

In [489]:
WindowVisibilityDataPivoted.head()

Empty DataFrame
Columns: [SubmittedDate, UserName, LISStoreCode]
Index: []

In [490]:
VisibilityColumns = [ col for col in WindowVisibilityDataPivoted.columns if "VisibilityAvailable" in col]
VisibilityColumns

[]

In [491]:
WindowVisibilityDataPivoted["Count_Of_Window_Yes"] = (
    WindowVisibilityDataPivoted[VisibilityColumns].apply(lambda x: x.astype(str).str.lower().eq("yes")).sum(axis=1))

In [492]:
WindowVisibilityDataPivoted.to_csv('WindowVisibilityDataPivoted.csv') #final

#### B. Bay Branding ( Paid Visibility Data Filter with POSMName as Bay Branding)

In [493]:
BayBrandingData = PaidVisibilityQuery[PaidVisibilityQuery['POSMName']=='Bay Branding']

In [494]:
BayBrandingData = BayBrandingData.drop(columns=['POSMName'])

In [495]:
SelectedDfBayBrandingData = select_columns(BayBrandingData, selected_columns)

In [496]:
ColumnsForPivotBB = SelectedDfBayBrandingData.copy()

In [497]:
SelectedDfBayBrandingDataUnpivotedDf = unpivot_other_columns(ColumnsForPivotBB, id_vars, attr_name="Attribute", val_name="Value")

In [498]:
SelectedDfBayBrandingDataUnpivotedDf["Brand_Attribute"] = (
    SelectedDfBayBrandingDataUnpivotedDf["POSMBrand"].astype(str) + "_" + SelectedDfBayBrandingDataUnpivotedDf["Attribute"].astype(str)
)


In [499]:
SelectedDfBayBrandingDataUnpivotedDf = SelectedDfBayBrandingDataUnpivotedDf.drop(columns = ['Attribute','POSMBrand']) 
group_cols = ["SubmittedDate","UserName","LISStoreCode","Brand_Attribute"]

SelectedDfBayBrandingDataUnpivotedDf_grouped = (
    SelectedDfBayBrandingDataUnpivotedDf
        .groupby(group_cols, as_index=False)["Value"]
        .first()
)

In [500]:
SelectedDfBayBrandingDataUnpivotedDf_grouped.head()

Empty DataFrame
Columns: [SubmittedDate, UserName, LISStoreCode, Brand_Attribute, Value]
Index: []

In [501]:
pivot_index = ["SubmittedDate", "UserName", "LISStoreCode"]

BayBrandingDataPivoted = (
    SelectedDfBayBrandingDataUnpivotedDf_grouped
        .pivot_table(
            index=pivot_index,
            columns="Brand_Attribute",
            values="Value",
            aggfunc="first"
        )
        .reset_index()
)

In [502]:
VisibilityColumns2 = [ col for col in BayBrandingDataPivoted.columns if "VisibilityAvailable" in col]
VisibilityColumns2

[]

In [503]:

BayBrandingDataPivoted["Count_Of_BayBranding_Yes"] = (
    BayBrandingDataPivoted[VisibilityColumns2].apply(lambda x: x.astype(str).str.lower().eq("yes")).sum(axis=1))

In [504]:
BayBrandingDataPivoted.to_csv('BayBrandingDataPivoted.csv')

In [505]:
BayBrandingDataPivoted.head() #final

Empty DataFrame
Columns: [SubmittedDate, UserName, LISStoreCode, Count_Of_BayBranding_Yes]
Index: []

#### Windows Visibility ( AuditInputExport)

In [506]:
WVisibilityAuditInputQuery= f"""
SELECT DISTINCT
    TRY_CONVERT(date, [SubmittedDate&Time], 105) AS [SubmittedDate&Time],
    VisitId,
    UserName,
    LISStoreCode,
    POSMBrand,
    PostCloseUpPhotoLink,
    PortalQuestions,
    PortalResponse
FROM PowerBI_LISV2.DaburPharmaManagement.VisibilityAuditInputReport
WHERE LOWER(UserName) NOT LIKE '%sup%'
  AND LOWER(UserName) NOT LIKE '%star%'
  AND PortalQuestions IS NOT NULL
  AND TRY_CONVERT(date, [SubmittedDate&Time], 105) BETWEEN '{start_date}' AND '{end_date}';"""

In [507]:
WVisibilityData = pd.read_sql(WVisibilityAuditInputQuery,conn)

In [508]:
WVisibilityData.to_csv('WVisibilityData.csv')

In [509]:
WVisibilityData.head()

SubmittedDate&Time  VisitId    UserName          LISStoreCode  \
0         2026-01-01   676440  DBR-ISP132   DBRPH-DEL0DL1238958   
1         2026-01-01   676443  DBR-ISP158  DBRPH-NGP0MH00169405   
2         2026-01-01   676444  DBR-ISP158  DBRPH-NGP0MH00169405   
3         2026-01-01   676444  DBR-ISP158  DBRPH-NGP0MH00169405   
4         2026-01-01   676449  DBR-ISP158  DBRPH-NGP0MH00551465   

             POSMBrand                               PostCloseUpPhotoLink  \
0  Dabur Amla Hair Oil  https://lisimagenewv3.loveinstore.com//LISImag...   
1                  OTC  https://lisimagenewv3.loveinstore.com//LISImag...   
2                  DCP  https://lisimagenewv3.loveinstore.com//LISImag...   
3                  DCP  https://lisimagenewv3.loveinstore.com//LISImag...   
4                  DCP  https://lisimagenewv3.loveinstore.com//LISImag...   

                                   PortalQuestions PortalResponse  
0  Planogram (placed left to right, Min 5 facings)            Yes  
1                                POSM availability             No  
2  Planogram (placed left to right, Min 5 facings)            Yes  
3                                POSM availability            Yes  
4  Planogram (placed left to right, Min 5 facings)            Yes

In [510]:
WVisibilityData['POSMBrand'] = WVisibilityData['POSMBrand'].replace({"DCP": "Dabur Chyawanprash","Dabur Amla Hair Oil": "Amla Hair Oil",
                                                                     "Red Tooth Paste": "ORAL CARE"})

In [511]:
WVisibilityData = WVisibilityData.rename(columns={"PostCloseUpPhotoLink": "URL"})


In [512]:
WVisibilityData_pivot = WVisibilityData.pivot_table(
    index=['VisitId','SubmittedDate&Time', 'UserName', 'LISStoreCode','POSMBrand','URL'],  # add identifiers as needed
    columns='PortalQuestions',
    values='PortalResponse',
    aggfunc='first'
).reset_index()
WVisibilityData_pivot = WVisibilityData_pivot.rename(columns={"Planogram (placed left to right, Min 5 facings)": "Planogram"})

In [513]:
WVisibilityData_pivot['WVAvailability'] = (
    (WVisibilityData_pivot['Planogram'].astype(str).str.lower() == "yes") &
    (WVisibilityData_pivot['POSM availability'].astype(str).str.lower() == "yes")
).map({True: "Yes", False: "No"})


In [514]:
WVisibilityData_pivot.head()

PortalQuestions  VisitId SubmittedDate&Time    UserName  \
0                 676431         2026-01-01  DBR-ISP199   
1                 676432         2026-01-01  DBR-ISP132   
2                 676433         2026-01-01  DBR-ISP132   
3                 676434         2026-01-01  DBR-ISP132   
4                 676436         2026-01-01  DBR-ISP132   

PortalQuestions                    LISStoreCode           POSMBrand  \
0                          DBRPH-MUM0MH00592694       Amla Hair Oil   
1                          DBRPH-DEL0DL11643699       Amla Hair Oil   
2                          DBRPH-DEL0DL11643699  Dabur Chyawanprash   
3                DBRPH-0022016394DEL0DL00087995  Dabur Chyawanprash   
4                          DBRPH-DEL0DL11642422       Amla Hair Oil   

PortalQuestions                                                URL  \
0                https://lisimagenewv3.loveinstore.com//LISImag...   
1                https://lisimagenewv3.loveinstore.com//LISImag...   
2                https://lisimagenewv3.loveinstore.com//LISImag...   
3                https://lisimagenewv3.loveinstore.com//LISImag...   
4                https://lisimagenewv3.loveinstore.com//LISImag...   

PortalQuestions POSM availability Planogram WVAvailability  
0                             Yes       Yes            Yes  
1                             Yes       Yes            Yes  
2                             Yes       Yes            Yes  
3                             Yes       Yes            Yes  
4                             Yes       Yes            Yes

In [515]:
WVisibilityData_pivot=WVisibilityData_pivot.drop(columns=['POSM availability','Planogram','VisitId'])

In [516]:
WVisibilityData_unpivot = WVisibilityData_pivot.melt(
    id_vars=["SubmittedDate&Time", "UserName", "LISStoreCode", "POSMBrand"],
    var_name="Attribute",
    value_name="Value"
)


In [517]:
WVisibilityData_unpivot['Brand_Attribute'] = WVisibilityData_unpivot['POSMBrand'].astype(str) + "_" + WVisibilityData_unpivot['Attribute'].astype(str)


In [518]:
WVisibilityData_unpivot=WVisibilityData_unpivot.drop(columns=['POSMBrand','Attribute'])

In [519]:
WVisibilityData_unpivot.head()

SubmittedDate&Time    UserName                    LISStoreCode  \
0         2026-01-01  DBR-ISP199            DBRPH-MUM0MH00592694   
1         2026-01-01  DBR-ISP132            DBRPH-DEL0DL11643699   
2         2026-01-01  DBR-ISP132            DBRPH-DEL0DL11643699   
3         2026-01-01  DBR-ISP132  DBRPH-0022016394DEL0DL00087995   
4         2026-01-01  DBR-ISP132            DBRPH-DEL0DL11642422   

                                               Value         Brand_Attribute  
0  https://lisimagenewv3.loveinstore.com//LISImag...       Amla Hair Oil_URL  
1  https://lisimagenewv3.loveinstore.com//LISImag...       Amla Hair Oil_URL  
2  https://lisimagenewv3.loveinstore.com//LISImag...  Dabur Chyawanprash_URL  
3  https://lisimagenewv3.loveinstore.com//LISImag...  Dabur Chyawanprash_URL  
4  https://lisimagenewv3.loveinstore.com//LISImag...       Amla Hair Oil_URL

In [520]:
group_cols = ["SubmittedDate&Time","UserName","LISStoreCode","Brand_Attribute"]

WVisibilityData_unpivot_grouped = (
    WVisibilityData_unpivot
        .groupby(group_cols, as_index=False)["Value"]
        .first()
)

In [521]:
WVisibilityData_unpivot_grouped.head()

SubmittedDate&Time    UserName                    LISStoreCode  \
0         2026-01-01  DBR-ISP132  DBRPH-0022016394DEL0DL00087995   
1         2026-01-01  DBR-ISP132  DBRPH-0022016394DEL0DL00087995   
2         2026-01-01  DBR-ISP132            DBRPH-DEL0DL11642422   
3         2026-01-01  DBR-ISP132            DBRPH-DEL0DL11642422   
4         2026-01-01  DBR-ISP132            DBRPH-DEL0DL11643699   

                     Brand_Attribute  \
0             Dabur Chyawanprash_URL   
1  Dabur Chyawanprash_WVAvailability   
2                  Amla Hair Oil_URL   
3       Amla Hair Oil_WVAvailability   
4                  Amla Hair Oil_URL   

                                               Value  
0  https://lisimagenewv3.loveinstore.com//LISImag...  
1                                                Yes  
2  https://lisimagenewv3.loveinstore.com//LISImag...  
3                                                Yes  
4  https://lisimagenewv3.loveinstore.com//LISImag...

In [522]:
pivot_index = ["SubmittedDate&Time", "UserName", "LISStoreCode"]

WVisibilityData_pivot = (
    WVisibilityData_unpivot_grouped
        .pivot_table(
            index=pivot_index,
            columns="Brand_Attribute",
            values="Value",
            aggfunc="first"
        )
        .reset_index()
)

In [523]:
WVisibilityData_pivot.head()

Brand_Attribute SubmittedDate&Time    UserName  \
0                       2026-01-01  DBR-ISP132   
1                       2026-01-01  DBR-ISP132   
2                       2026-01-01  DBR-ISP132   
3                       2026-01-01  DBR-ISP132   
4                       2026-01-01  DBR-ISP158   

Brand_Attribute                    LISStoreCode  \
0                DBRPH-0022016394DEL0DL00087995   
1                          DBRPH-DEL0DL11642422   
2                          DBRPH-DEL0DL11643699   
3                           DBRPH-DEL0DL1238958   
4                          DBRPH-NGP0MH00169405   

Brand_Attribute                                  Amla Hair Oil_URL  \
0                                                              NaN   
1                https://lisimagenewv3.loveinstore.com//LISImag...   
2                https://lisimagenewv3.loveinstore.com//LISImag...   
3                https://lisimagenewv3.loveinstore.com//LISImag...   
4                                                              NaN   

Brand_Attribute Amla Hair Oil_WVAvailability  \
0                                        NaN   
1                                        Yes   
2                                        Yes   
3                                        Yes   
4                                        NaN   

Brand_Attribute                             Dabur Chyawanprash_URL  \
0                https://lisimagenewv3.loveinstore.com//LISImag...   
1                                                              NaN   
2                https://lisimagenewv3.loveinstore.com//LISImag...   
3                                                              NaN   
4                https://lisimagenewv3.loveinstore.com//LISImag...   

Brand_Attribute Dabur Chyawanprash_WVAvailability Honey_URL  \
0                                             Yes       NaN   
1                                             NaN       NaN   
2                                             Yes       NaN   
3                                             NaN       NaN   
4                                             Yes       NaN   

Brand_Attribute Honey_WVAvailability ORAL CARE_URL ORAL CARE_WVAvailability  \
0                                NaN           NaN                      NaN   
1                                NaN           NaN                      NaN   
2                                NaN           NaN                      NaN   
3                                NaN           NaN                      NaN   
4                                NaN           NaN                      NaN   

Brand_Attribute                                            OTC_URL  \
0                                                              NaN   
1                                                              NaN   
2                                                              NaN   
3                                                              NaN   
4                https://lisimagenewv3.loveinstore.com//LISImag...   

Brand_Attribute OTC_WVAvailability  
0                              NaN  
1                              NaN  
2                              NaN  
3                              NaN  
4                               No

In [524]:
VisibilityColumns3 = [ col for col in WVisibilityData_pivot.columns if "WVAvailability" in col]
VisibilityColumns3

['Amla Hair Oil_WVAvailability',
 'Dabur Chyawanprash_WVAvailability',
 'Honey_WVAvailability',
 'ORAL CARE_WVAvailability',
 'OTC_WVAvailability']

In [525]:

WVisibilityData_pivot["Count_Of_WV_Yes"] = (
    WVisibilityData_pivot[VisibilityColumns3].apply(lambda x: x.astype(str).str.lower().eq("yes")).sum(axis=1))

In [526]:
WVisibilityData_pivot.head()

Brand_Attribute SubmittedDate&Time    UserName  \
0                       2026-01-01  DBR-ISP132   
1                       2026-01-01  DBR-ISP132   
2                       2026-01-01  DBR-ISP132   
3                       2026-01-01  DBR-ISP132   
4                       2026-01-01  DBR-ISP158   

Brand_Attribute                    LISStoreCode  \
0                DBRPH-0022016394DEL0DL00087995   
1                          DBRPH-DEL0DL11642422   
2                          DBRPH-DEL0DL11643699   
3                           DBRPH-DEL0DL1238958   
4                          DBRPH-NGP0MH00169405   

Brand_Attribute                                  Amla Hair Oil_URL  \
0                                                              NaN   
1                https://lisimagenewv3.loveinstore.com//LISImag...   
2                https://lisimagenewv3.loveinstore.com//LISImag...   
3                https://lisimagenewv3.loveinstore.com//LISImag...   
4                                                              NaN   

Brand_Attribute Amla Hair Oil_WVAvailability  \
0                                        NaN   
1                                        Yes   
2                                        Yes   
3                                        Yes   
4                                        NaN   

Brand_Attribute                             Dabur Chyawanprash_URL  \
0                https://lisimagenewv3.loveinstore.com//LISImag...   
1                                                              NaN   
2                https://lisimagenewv3.loveinstore.com//LISImag...   
3                                                              NaN   
4                https://lisimagenewv3.loveinstore.com//LISImag...   

Brand_Attribute Dabur Chyawanprash_WVAvailability Honey_URL  \
0                                             Yes       NaN   
1                                             NaN       NaN   
2                                             Yes       NaN   
3                                             NaN       NaN   
4                                             Yes       NaN   

Brand_Attribute Honey_WVAvailability ORAL CARE_URL ORAL CARE_WVAvailability  \
0                                NaN           NaN                      NaN   
1                                NaN           NaN                      NaN   
2                                NaN           NaN                      NaN   
3                                NaN           NaN                      NaN   
4                                NaN           NaN                      NaN   

Brand_Attribute                                            OTC_URL  \
0                                                              NaN   
1                                                              NaN   
2                                                              NaN   
3                                                              NaN   
4                https://lisimagenewv3.loveinstore.com//LISImag...   

Brand_Attribute OTC_WVAvailability  Count_Of_WV_Yes  
0                              NaN                1  
1                              NaN                1  
2                              NaN                2  
3                              NaN                1  
4                               No                1

In [527]:
WVisibilityData_pivot = WVisibilityData_pivot[["SubmittedDate&Time", "UserName", "LISStoreCode", "Count_Of_WV_Yes",
                                               "Honey_WVAvailability", "Honey_URL", "Dabur Chyawanprash_WVAvailability", "Dabur Chyawanprash_URL", 
                                               "Amla Hair Oil_WVAvailability", "Amla Hair Oil_URL", 
                                               "OTC_WVAvailability", "OTC_URL", "ORAL CARE_WVAvailability", "ORAL CARE_URL"]]

In [528]:
WVisibilityData_pivot.head()

Brand_Attribute SubmittedDate&Time    UserName  \
0                       2026-01-01  DBR-ISP132   
1                       2026-01-01  DBR-ISP132   
2                       2026-01-01  DBR-ISP132   
3                       2026-01-01  DBR-ISP132   
4                       2026-01-01  DBR-ISP158   

Brand_Attribute                    LISStoreCode  Count_Of_WV_Yes  \
0                DBRPH-0022016394DEL0DL00087995                1   
1                          DBRPH-DEL0DL11642422                1   
2                          DBRPH-DEL0DL11643699                2   
3                           DBRPH-DEL0DL1238958                1   
4                          DBRPH-NGP0MH00169405                1   

Brand_Attribute Honey_WVAvailability Honey_URL  \
0                                NaN       NaN   
1                                NaN       NaN   
2                                NaN       NaN   
3                                NaN       NaN   
4                                NaN       NaN   

Brand_Attribute Dabur Chyawanprash_WVAvailability  \
0                                             Yes   
1                                             NaN   
2                                             Yes   
3                                             NaN   
4                                             Yes   

Brand_Attribute                             Dabur Chyawanprash_URL  \
0                https://lisimagenewv3.loveinstore.com//LISImag...   
1                                                              NaN   
2                https://lisimagenewv3.loveinstore.com//LISImag...   
3                                                              NaN   
4                https://lisimagenewv3.loveinstore.com//LISImag...   

Brand_Attribute Amla Hair Oil_WVAvailability  \
0                                        NaN   
1                                        Yes   
2                                        Yes   
3                                        Yes   
4                                        NaN   

Brand_Attribute                                  Amla Hair Oil_URL  \
0                                                              NaN   
1                https://lisimagenewv3.loveinstore.com//LISImag...   
2                https://lisimagenewv3.loveinstore.com//LISImag...   
3                https://lisimagenewv3.loveinstore.com//LISImag...   
4                                                              NaN   

Brand_Attribute OTC_WVAvailability  \
0                              NaN   
1                              NaN   
2                              NaN   
3                              NaN   
4                               No   

Brand_Attribute                                            OTC_URL  \
0                                                              NaN   
1                                                              NaN   
2                                                              NaN   
3                                                              NaN   
4                https://lisimagenewv3.loveinstore.com//LISImag...   

Brand_Attribute ORAL CARE_WVAvailability ORAL CARE_URL  
0                                    NaN           NaN  
1                                    NaN           NaN  
2                                    NaN           NaN  
3                                    NaN           NaN  
4                                    NaN           NaN

#### Merging Data
##### Merging BayBranding

In [529]:
#kunal

In [530]:
BayBrandingDataPivoted = BayBrandingDataPivoted.add_prefix("BB_") # renaming column to add prefix

In [531]:
BayBrandingDataPivoted['BB_SubmittedDate'] = pd.to_datetime(BayBrandingDataPivoted['BB_SubmittedDate'], errors='coerce', dayfirst=True).dt.normalize()

In [532]:
pss3 = pd.merge(pss2,BayBrandingDataPivoted, left_on = ['LISStoreCode','UserName','VisitDate'],
                right_on =['BB_LISStoreCode','BB_UserName','BB_SubmittedDate'],how ='left')

In [533]:
# pss3.head()

##### Merging Paid Visibility

In [534]:
#KUNAL

In [535]:
WindowVisibilityDataPivoted['SubmittedDate'] = pd.to_datetime(WindowVisibilityDataPivoted['SubmittedDate'], errors='coerce', dayfirst=True).dt.normalize()

In [536]:
WindowVisibilityDataPivoted = WindowVisibilityDataPivoted.add_prefix("PWV_")

In [537]:
# WindowVisibilityDataPivoted = WindowVisibilityDataPivoted.rename(columns={"SubmittedDate": "PWV_SubmittedDate"}) # renmamed column to avoid duplicacy

In [538]:
pss4 = pd.merge(pss3,WindowVisibilityDataPivoted, left_on = ['LISStoreCode','UserName','VisitDate'],
                right_on =['PWV_LISStoreCode','PWV_UserName','PWV_SubmittedDate'],how ='left')

In [539]:
# pss4.to_csv('pss4.csv')

##### Windows WVisibility ( AuditInputExport)

In [540]:
# WVisibilityData_pivot #Table

In [541]:
WVisibilityData_pivot = WVisibilityData_pivot.rename(columns={"SubmittedDate&Time": "WVAudirSubmittedDate"}) # renmamed column to avoid duplicacy
WVisibilityData_pivot['WVAudirSubmittedDate'] = pd.to_datetime(WVisibilityData_pivot['WVAudirSubmittedDate'], errors='coerce', dayfirst=True).dt.normalize()

In [542]:
WVisibilityData_pivot = WVisibilityData_pivot.add_prefix("WV_")


In [543]:
WVisibilityData_pivot.head()

Brand_Attribute WV_WVAudirSubmittedDate WV_UserName  \
0                            2026-01-01  DBR-ISP132   
1                            2026-01-01  DBR-ISP132   
2                            2026-01-01  DBR-ISP132   
3                            2026-01-01  DBR-ISP132   
4                            2026-01-01  DBR-ISP158   

Brand_Attribute                 WV_LISStoreCode  WV_Count_Of_WV_Yes  \
0                DBRPH-0022016394DEL0DL00087995                   1   
1                          DBRPH-DEL0DL11642422                   1   
2                          DBRPH-DEL0DL11643699                   2   
3                           DBRPH-DEL0DL1238958                   1   
4                          DBRPH-NGP0MH00169405                   1   

Brand_Attribute WV_Honey_WVAvailability WV_Honey_URL  \
0                                   NaN          NaN   
1                                   NaN          NaN   
2                                   NaN          NaN   
3                                   NaN          NaN   
4                                   NaN          NaN   

Brand_Attribute WV_Dabur Chyawanprash_WVAvailability  \
0                                                Yes   
1                                                NaN   
2                                                Yes   
3                                                NaN   
4                                                Yes   

Brand_Attribute                          WV_Dabur Chyawanprash_URL  \
0                https://lisimagenewv3.loveinstore.com//LISImag...   
1                                                              NaN   
2                https://lisimagenewv3.loveinstore.com//LISImag...   
3                                                              NaN   
4                https://lisimagenewv3.loveinstore.com//LISImag...   

Brand_Attribute WV_Amla Hair Oil_WVAvailability  \
0                                           NaN   
1                                           Yes   
2                                           Yes   
3                                           Yes   
4                                           NaN   

Brand_Attribute                               WV_Amla Hair Oil_URL  \
0                                                              NaN   
1                https://lisimagenewv3.loveinstore.com//LISImag...   
2                https://lisimagenewv3.loveinstore.com//LISImag...   
3                https://lisimagenewv3.loveinstore.com//LISImag...   
4                                                              NaN   

Brand_Attribute WV_OTC_WVAvailability  \
0                                 NaN   
1                                 NaN   
2                                 NaN   
3                                 NaN   
4                                  No   

Brand_Attribute                                         WV_OTC_URL  \
0                                                              NaN   
1                                                              NaN   
2                                                              NaN   
3                                                              NaN   
4                https://lisimagenewv3.loveinstore.com//LISImag...   

Brand_Attribute WV_ORAL CARE_WVAvailability WV_ORAL CARE_URL  
0                                       NaN              NaN  
1                                       NaN              NaN  
2                                       NaN              NaN  
3                                       NaN              NaN  
4                                       NaN              NaN

In [544]:
pss5 = pd.merge(pss4,WVisibilityData_pivot, left_on = ['LISStoreCode','UserName','VisitDate'],
                right_on =['WV_LISStoreCode','WV_UserName','WV_WVAudirSubmittedDate'],how ='left')

In [545]:
pss5.head()

Empty DataFrame
Columns: [UserName, FullName, LISStoreCode, StoreName, VisitDate, PJPStatus, CheckIn-Time, CheckOut-Time, TimeDuration, AutoCheckOutStatus, NoPermissionReason, StoreCheck-InImageLink, StoreCheck-OutImageLink, FR Image, NoPermissionImageLink, AttendanceType, Covered, Merchandised, PS_SubmittedDate, PS_Username, PS_LISStoreCode, PS_OSA Target, PS_OSA Avail, SOS_SubmittedDate, SOS_UserName, SOS_LISStoreCode, SOS_SumSOSAchieved, SOS_SumIsCategoryAvailable, BB_SubmittedDate, BB_UserName, BB_LISStoreCode, BB_Count_Of_BayBranding_Yes, PWV_SubmittedDate, PWV_UserName, PWV_LISStoreCode, PWV_Count_Of_Window_Yes, WV_WVAudirSubmittedDate, WV_UserName, WV_LISStoreCode, WV_Count_Of_WV_Yes, WV_Honey_WVAvailability, WV_Honey_URL, WV_Dabur Chyawanprash_WVAvailability, WV_Dabur Chyawanprash_URL, WV_Amla Hair Oil_WVAvailability, WV_Amla Hair Oil_URL, WV_OTC_WVAvailability, WV_OTC_URL, WV_ORAL CARE_WVAvailability, WV_ORAL CARE_URL]
Index: []

[0 rows x 50 columns]

In [546]:
# pss5.to_csv('pss5.csv')

#### StoreMaster
Store Master is maintained on GoogleSheet By MIS & it is extracted from there and merged with KPI data

In [547]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc

from datetime import datetime


In [548]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [549]:
# Open Sheet
sheet = client.open_by_key("1eGDxXlWAiORbLQPrWHKSq78HLB68ZTTIkPbB4RazCmk") # Master Sheets ( Dimensions Sheet)

In [550]:
StoreMaster = sheet.worksheet("StoreMaster")
StoreMaster = get_as_dataframe(StoreMaster)

In [551]:
StoreMaster.head(2)

Region   State         City        StoreType     Chain      Club  \
0   East  Orissa  Bhubaneswar  Migrated MT OFO  DBRMTOFO  Platinum   
1   East  Orissa  Bhubaneswar  Migrated MT OFO  DBRMTOFO  Platinum   

          BB/WV Format Client Store Code          LISStoreCode  \
0  Bay Branding     GT    BBR0OR11499590  DBRPH-BBR0OR11499590   
1  Bay Branding     GT    BBR0OR01634099  DBRPH-BBR0OR01634099   

                         StoreName SupervisorName    UserName  \
0  NOW GROCCERY (HINDUSTAN AGENCY)     Soma Maiti  DBR-ISP313   
1            SURUCHI BAZAR PVT LTD     Soma Maiti  DBR-ISP313   

         EmployeeName Designation StoreStatus EmployeeStatus    Class  
0  Satyaprakash Samal          MR      Active         Active  Class C  
1  Satyaprakash Samal          MR      Active         Active  Class C

In [552]:
cols_to_exclude = ['StoreName', 'UserName']   # columns in pss5 you don't want

pss5_filtered = pss5.drop(columns=cols_to_exclude)
pss6  = pd.merge(StoreMaster,pss5_filtered,on = 'LISStoreCode',how='left')

In [553]:
pss6= pss6.fillna("")
pss6.head(2)

Region   State         City        StoreType     Chain      Club  \
0   East  Orissa  Bhubaneswar  Migrated MT OFO  DBRMTOFO  Platinum   
1   East  Orissa  Bhubaneswar  Migrated MT OFO  DBRMTOFO  Platinum   

          BB/WV Format Client Store Code          LISStoreCode  ...  \
0  Bay Branding     GT    BBR0OR11499590  DBRPH-BBR0OR11499590  ...   
1  Bay Branding     GT    BBR0OR01634099  DBRPH-BBR0OR01634099  ...   

  WV_Honey_WVAvailability WV_Honey_URL WV_Dabur Chyawanprash_WVAvailability  \
0                                                                             
1                                                                             

  WV_Dabur Chyawanprash_URL WV_Amla Hair Oil_WVAvailability  \
0                                                             
1                                                             

  WV_Amla Hair Oil_URL WV_OTC_WVAvailability WV_OTC_URL  \
0                                                         
1                                                         

  WV_ORAL CARE_WVAvailability WV_ORAL CARE_URL  
0                                               
1                                               

[2 rows x 65 columns]

In [554]:
pss6.to_csv('pss6.csv')